In [ ]:
# Monthly average electricity consumption + temperture

avg_t_mean = df.groupby('Month')['Avg Temperature (Celsius)'].mean()
avg_pc_mean = df.groupby('Month')['Avg Electricity Consumption per Household (kWh)'].mean()

fig = plt.figure(figsize=(20,10))
fig.set_facecolor('white') 
ax = fig.add_subplot()
 
ax.bar(avg_t_mean.index, avg_t_mean , label='Avg Temperature (Celsius)')
ax.set_ylabel('Avg Temperature (Celsius)', fontsize=14, color='black') 
ax.set_xlabel('Month', fontsize=14, color='black')
 
ax2 = ax.twinx()
ax2.set_ylabel('Avg Electricity Consumption per Household (kWh)', fontsize=14, color='red')
ax2.plot(avg_pc_mean.index ,avg_pc_mean, label='Avg Electricity Consumption per Household (kWh)',color='red', linewidth = 5)

plt.xticks(np.arange(0,13,1),rotation=0)
plt.title('Avg Temperature & Avg Electricity Consumption per Household (kWh)',fontsize=20)
plt.legend(loc = 2, fontsize=20, frameon=True, shadow=True)

plt.show()

In [ ]:
# # #Year per: 84 Month per: 70 Region per: 140 Total data:840
# 
# monthly_data = df.groupby('Region')['Avg Electricity Consumption per Household (kWh) Household (kWh)'].mean( ).reset_index()
# df_mean=df.groupby(['YeaAvg Electricity Consumption per Household (kWh)r Househol# d (kWh)']).mean()
# unstacked_df = df_mean.unst# ack(level='Year')
# fig, ax_1 = plt.subplot# s(figsize=(12, 8))
# ax_1.set# _xlabel('Region')
# ax_1.set_xticks# (monthly_data.indAvg Electricity Consumption per Household (kWh)on # per Household (kWh)')
# ax_1.plot(monthly_data['LocAvg Electricity Consumption per Household (kWh)'on per Household (kWh)'], color='orange',mark# er='o', ms=4,alpha=1)

# ax_1.tick_params(axis='# y', labelcolor='orange')
# ### #####
# ax_2 = ax_Small Pan Evaporation (mm)('S# mall Pan Evaporation (mm)')
# unstacked_df.plot(k# ind='bar',ax=ax_2,alpha=0.5)
# ax_2.tick_params(# axis='y', labelcolor='blue')

# plt.title(f'Avg Electricity Consumption per Household#  (kWh) from#  Region,Region'# )
# plt.grid()
# plt.tight_layout()
# for x_val,y_val in zip(monthly_data['Region'],monthly_data['Avg Electricity C# onsumption per Household (kWh)']):
# 	ax_1.text(x_val, y_val +0.1,f'{y_val:.2f}',ha='center',va='bottom'# ,fontsize=9,color='black',zorde# r=5)
# plt.subplots_adjust(right=1.8)
# plt.show()

In [ ]:
def evaluation(models: List[Union[GridSearchCV, BaseEstimator]], X_train_list: List[pd.DataFrame], X_test_list: List[pd.DataFrame], y_train: pd.Series, y_test: pd.Series) -> pd.DataFrame:
	"""
	Evaluates a list of models (either GridSearchCV objects or regular estimators)
	on training and test data, and returns a summary DataFrame of evaluation metrics.

	Parameters:
		models (List[Union[GridSearchCV, BaseEstimator]]): List of models to evaluate.
		X_train_list (List[pd.DataFrame]): Training features.
		X_test_list (List[pd.DataFrame]): Testing features.
		y_train (pd.Series): Training labels.
		y_test (pd.Series): Testing labels.

	Returns:
		pd.DataFrame: A summary table with model name, evaluation metrics,
					  training time, and best hyperparameters (if available).
	"""
	results = []
	
	for i, (X_train, X_test) in enumerate(zip(X_train_list, X_test_list)):
		for model in models:
			# Training for each X_train
			model_clone = clone(model)
			model_clone.fit(X_train, y_train)
			
			# Check if a model is a GridSearchCV object
			if isinstance(model_clone, GridSearchCV):
				best_model = model_clone.best_estimator_
				best_hyperparams = model_clone.best_params_
			else:
				best_model = model
				best_hyperparams = 'N/A'
			model_name = str(best_model.__class__.__name__)
			
			# Testing for each X_test
			y_pred = best_model.predict(X_test)
			
			# Evaluation
			mae = mean_absolute_error(y_test,y_pred)
			mse = mean_squared_error(y_test, y_pred)
			rmse = np.sqrt(mse)
			r2 = r2_score(y_test,y_pred)
			adj_r2 = adj_r2_score(y_test,y_pred,X_test)
			
			# Result dataframe
			result = pd.DataFrame({'Feature Index': [i],
								   'Model': [model_name],
								   'MAE': [mae],
								   'RMSE': [rmse],
								   'R^2': [r2],
								   'Adj-R^2': [adj_r2],
								   'Training time': [training_time],
								   'Best hyperparameters': [best_hyperparams]})
			results.append(result)
			
		# Sort results by Model
		results = pd.concat(results).sort_values(by=['Feature Index', 'Model']).reset_index(drop=True)

	return results

models = [model_lr, model_en, model_dt, model_svr, model_rf, model_ada, model_xgb, model_stacking]
results = evaluation(models=models, X_train=X_train, X_test=X_test, y_train=y_train, y_test=y_test)
results

In [ ]:
X_trains=[X1_train,X2_train,X3_train,X4_train,X5_train]
X_tests=[X1_test,X2_test,X3_test,X4_test,X5_test]
dummy_prefix = 'Location'
dummy_sep = '_'
for i, (train_df, test_df) in enumerate(zip(X_trains, X_tests)):
    print(f"  처리 중: {i+1}번째 Train/Test 쌍") # i+1 하면 1, 2, 3, 4, 5 가 됩니다.
    train_name=f'X{i+1}_train'
    test_name=f'X{i+1}_test'
    train_processed = pd.get_dummies(train_df, columns=['Location'],dtype=int, prefix=dummy_prefix, prefix_sep=dummy_sep)
    test_processed = pd.get_dummies(test_df, columns=['Location'],dtype=int, prefix=dummy_prefix, prefix_sep=dummy_sep)
    all_cols = list(set(train_processed.columns) | set(test_processed.columns))
    train_reindexed = train_processed.reindex(columns=all_cols, fill_value=0)
    test_reindexed = test_processed.reindex(columns=all_cols, fill_value=0)
    # 원-핫 인코딩 시 사용한 prefix를 기준으로 더미 컬럼 식별
    dummy_cols = [col for col in train_reindexed.columns if col.startswith(f"{dummy_prefix}{dummy_sep}")]
    # 더미 컬럼을 제외한 나머지를 스케일링 대상 숫자형 컬럼으로 간주
    numerical_cols = [col for col in train_reindexed.columns if col not in dummy_cols]
    scaler=StandardScaler()
    scaler.fit(train_reindexed[numerical_cols])
    scaled_train_numerical_np = scaler.transform(train_reindexed[numerical_cols])
    train_scaled_numerical_df = pd.DataFrame(scaled_train_numerical_np,
                                             index=train_reindexed.index,
                                             columns=numerical_cols)

    scaled_test_numerical_np = scaler.transform(test_reindexed[numerical_cols])
    test_scaled_numerical_df = pd.DataFrame(scaled_test_numerical_np,
                                            index=test_reindexed.index,
                                            columns=numerical_cols)
    final_train_df = pd.concat([train_scaled_numerical_df, train_reindexed[dummy_cols]], axis=1)
    final_test_df = pd.concat([test_scaled_numerical_df, test_reindexed[dummy_cols]], axis=1)

    print(f"    스케일링 대상 컬럼 (Numerical): {numerical_cols}")
    print(f"    유지 대상 컬럼 (Dummy): {dummy_cols}")
    globals()[train_name] = final_train_df
    globals()[test_name] = final_test_df